Test CNN on spectrogram

In [186]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
import time

In [187]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [188]:
#The compose function allows for multiple transforms
#transforms.ToTensor() converts our PILImage to a tensor of shape (C x H x W) in the range [0,1]
#transforms.Normalize(mean,std) normalizes a tensor to a (mean, std) for (R, G, B)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./cifardata', train=True, download=True, transform=transform)

test_set = torchvision.datasets.CIFAR10(root='./cifardata', train=False, download=True, transform=transform)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [189]:
#Training
n_training_samples = 20000
train_sampler = SubsetRandomSampler(np.arange(n_training_samples, dtype=np.int64))

#Validation
n_val_samples = 5000
val_sampler = SubsetRandomSampler(np.arange(n_training_samples, n_training_samples + n_val_samples, dtype=np.int64))

#Test
n_test_samples = 5000
test_sampler = SubsetRandomSampler(np.arange(n_test_samples, dtype=np.int64))

In [190]:
class SimpleCNN(torch.nn.Module):
    
    #Our batch shape for input x is (3, 32, 32)
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        #Input channels = 3, output channels = 18
        self.conv1 = torch.nn.Conv2d(1, 18, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        #4608 input features, 64 output features (see sizing flow below)
        self.fc1 = torch.nn.Linear(18 * 16 * 16, 64)
        
        #64 input features, 10 output features for our 10 defined classes
        self.fc2 = torch.nn.Linear(64, 10)
        
    def forward(self, x):
        
        #Computes the activation of the first convolution
        #Size changes from (3, 32, 32) to (18, 32, 32)
        x = F.relu(self.conv1(x))
        
        #Size changes from (18, 32, 32) to (18, 16, 16)
        x = self.pool(x)
        
        #Reshape data to input to the input layer of the neural net
        #Size changes from (18, 16, 16) to (1, 4608)
        #Recall that the -1 infers this dimension from the other given dimension
        x = x.view(-1, 18 * 16 *16)
        
        #Computes the activation of the first fully connected layer
        #Size changes from (1, 4608) to (1, 64)
        x = F.relu(self.fc1(x))
        
        #Computes the second fully connected layer (activation applied later)
        #Size changes from (1, 64) to (1, 10)
        x = self.fc2(x)
        return(x)

loss = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

print(SimpleCNN())

SimpleCNN(
  (conv1): Conv2d(1, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4608, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)


In [196]:
#DataLoader takes in a dataset and a sampler for loading (num_workers deals with system level memory) 
def get_train_loader(batch_size):
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                           sampler=train_sampler, num_workers=2)
    return(train_loader)

#Test and validation loaders have constant batch sizes, so we can define them directly
test_loader = torch.utils.data.DataLoader(test_set, batch_size=4, sampler=test_sampler, num_workers=2)
val_loader = torch.utils.data.DataLoader(train_set, batch_size=128, sampler=val_sampler, num_workers=2)

def trainNet(net, batch_size, n_epochs, learning_rate):
    
    #Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    #Get training data
    train_loader = get_train_loader(batch_size)
    n_batches = len(train_loader)
    
    #Create our loss and optimizer functions
    loss = torch.nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(train_loader, 0):
            
            #Get inputs
            inputs, labels = data
            
            # One color channel in
            inputs = inputs[:,0,:,:]
            inputs = torch.unsqueeze(inputs,1)
            inputs = inputs.view(32,1,64,16)
            print(inputs.size(), inputs.type())
            print(labels.size(), labels.type())
            
            #Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.data[0]
            total_train_loss += loss_size.data[0]
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        for inputs, labels in val_loader:
            
            #Wrap tensors in Variables
            inputs, labels = Variable(inputs), Variable(labels)
            
            # One color channel in
            inputs = inputs[:,0,:,:]
            inputs = torch.unsqueeze(inputs,1)
            
            #Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels)
            total_val_loss += val_loss_size.data[0]
            
        print("Validation loss = {:.2f}".format(total_val_loss / len(val_loader)))
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

In [197]:
CNN = SimpleCNN()
trainNet(CNN, batch_size=32, n_epochs=5, learning_rate=0.001)

===== HYPERPARAMETERS =====
batch_size= 32
epochs= 5
learning_rate= 0.001
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor


/home/Christian/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:64: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/home/Christian/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:65: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatT

torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatT

torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
Epoch 1, 40% 	 train_loss: 2.01 took: 0.57s
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTen

Process Process-227:
Process Process-228:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/Christian/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/Christian/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/Christian/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)


torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor
torch.Size([32, 1, 64, 16]) torch.FloatTensor
torch.Size([32]) torch.LongTensor


  File "/home/Christian/anaconda2/envs/py36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/Christian/anaconda2/envs/py36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _worker_loop
    r = index_queue.get()
  File "/home/Christian/anaconda2/envs/py36/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 61, in _worker_loop
    data_queue.put((idx, samples))
  File "/home/Christian/anaconda2/envs/py36/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/Christian/anaconda2/envs/py36/lib/python3.6/multiprocessing/queues.py", line 341, in put
    obj = _ForkingPickler.dumps(obj)
  File "/home/Christian/anaconda2/envs/py36/lib/python3.6/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/home/Christian/anaconda2/envs/py36/lib/python3.6/multiprocessing/reduction.py", line 51, i

KeyboardInterrupt: 

KeyboardInterrupt


In [185]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data          
        images = images[:,0,:,:]
        images = torch.unsqueeze(images,1)
        outputs = CNN(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 53 %
